IMPORT

In [ ]:
import pandas as pd
import numpy as np
import os
import Levenshtein as lev
from rapidfuzz import fuzz
import re
from pandas import Timestamp
from matplotlib import pyplot as plt

FUNCTIONS   

In [ ]:
def pulisci_stringhe_fuzz(df_temp, colonna, soglia_fuzz):
    df_temp2=df_temp.copy() #poi usare df_temp2
    colonna_mod = f'{colonna}_mod'
    df_temp2[colonna_mod] = df_temp2[colonna].str.upper()


    prima= set(df_temp2[colonna_mod].unique())
    conto1=len(prima)

    diz = df_temp2[colonna_mod].value_counts().to_dict()


    for val1 in diz:
        for val2 in diz:
            if val1 != val2:
                similarità = fuzz.ratio(val1, val2)
                if similarità >= soglia_fuzz and diz[val1] >= diz[val2]:
                    df_temp2.loc[df_temp2[colonna_mod] == val2, colonna_mod] = val1  #Sostituisci tutti i valori uguali a val2 nella colonna colonna_mod del DataFrame df_temp2 con val1.# creare nuova colonna invece di replace
                elif similarità >=soglia_fuzz and diz[val1]<diz[val2]:
                    df_temp2.loc[df_temp2[colonna_mod] == val1, colonna_mod] = val2

   
    dopo= set(df_temp2[colonna_mod].unique())
    conto2=len(dopo)
    mancanti = prima - dopo
    print("Valori rimossi o modificati:", mancanti, len(mancanti))

    varianti_dict = {
    key: sorted(set(s.upper() for s in group[colonna].unique() if isinstance(s, str)))
    for key, group in df_temp2.groupby(colonna_mod)}

    dict_ordinato=sorted(varianti_dict.items(), key=lambda couple: len(couple[1]), reverse=True)
     

    return df_temp2, varianti_dict #,dftemp2[colonna_mod],conto1, conto2, mancanti, len(mancanti)

In [ ]:
def pulisci_stringhe_lev(df_temp,colonna,soglia_lev):
    df_temp2=df_temp.copy()
    colonna_mod = f'{colonna}_mod'
    df_temp2[colonna_mod] = df_temp2[colonna].str.upper()
    

    prima= set(df_temp2[colonna_mod].unique())
    conto1=len(prima)
    
    diz= df_temp2[colonna_mod].value_counts().to_dict()


    for val1 in diz:
        for val2 in diz:
            if val1!=val2:
                distanza=lev.distance(val1,val2)
                if distanza <= soglia_lev and diz[val1]>=diz[val2]:
                    df_temp2.loc[df_temp2[colonna_mod] == val2, colonna_mod] = val1
                elif distanza <= soglia_lev and diz[val1]<diz[val2]:
                    df_temp2.loc[df_temp2[colonna_mod] == val1, colonna_mod] = val2

    
    
    dopo= set(df_temp2[colonna_mod].unique())
    conto2=len(dopo)
    mancanti = prima - dopo
    print("Valori rimossi o modificati:", mancanti, len(mancanti))

    dict = {
    key: sorted(set(s.upper() for s in group[colonna].unique() if isinstance(s, str)))
    for key, group in df_temp2.groupby(colonna_mod)}

    dict_ordinato=sorted(dict.items(), key=lambda couple: len(couple[1]), reverse=True)

    return df_temp2, dict #, conto1, conto2, mancanti, len(mancanti)



In [ ]:
def pulisci_trattino(val):
    val = str(val)
    # Se c'è un "-" in mezzo, lo sostituisce con un "."
    val = re.sub(r'(?<!^)-', '.', val) #(?<!^) è una negative lookbehind assertion: controlla che prima del trattino non ci sia l'inizio della stringa (^)
    return val

In [ ]:
def creo_dict(df,colonna_originale,colonna_modificata):
    dizionario={key: list(group[colonna_originale].unique())
    for key, group in df.groupby(colonna_modificata)}
    return dizionario

In [ ]:
def rendo_upper(df,lista_varianti):
    #RENDO LE COLONNE MAIUSCOLE, COSì DA ELIMINARE I DUPLICATI

    df_upper= df.copy()
    for col in lista_varianti:
        df_upper[col] = df_upper[col].apply(
            lambda x: sorted(set(s.upper() for s in x)) if isinstance(x, list) else x
    )
    return df_upper

GET DATA

In [ ]:
# Leggi tutti i fogli come dizionario di DataFrame
xls = pd.ExcelFile("../data/Superstore_mod_v1.xlsx")
df_dict = pd.read_excel(xls, sheet_name=None)  # None legge tutti i fogli

# df_dict è un dizionario con chiavi = nomi dei fogli, valori = DataFrame
for sheet_name, df in df_dict.items():
    print(f"Foglio: {sheet_name}")
    #display(df.head())  
    globals()[f'df_{sheet_name.lower()}'] = df  # globals() in Python serve a restituire un dizionario contenente tutte le variabili globali 
                                                #attualmente disponibili nel contesto in cui viene chiamato

display(df_countries)

FOGLIO COUNTRIES

In [ ]:
'''COLONNA STATI'''
df_countries_raw=df_countries.drop_duplicates() #tolgo i duplicati da tutto il df_countries
#print(df_countries_raw['State'].sort_values().unique())


df_countries_fuzz_90, dict_countries_fuzz_90= pulisci_stringhe_fuzz(df_countries_raw,'State',90) 
df_countries_fuzz_90 = df_countries_fuzz_90.rename(columns={'State_mod': 'State_fuzz_90'})
df_countries_fuzz_90['State_fuzz_90'] = df_countries_fuzz_90['State_fuzz_90'].replace({
    'NY': 'NEW YORK', 
    'NC': 'NORTH CAROLINA'
})
display(df_countries_fuzz_90)
print(dict_countries_fuzz_90)

df_stati_corretti_fuzz_90=pd.DataFrame([(k, v) for k, v in dict_countries_fuzz_90.items() if len(v)>1], columns=['State', 'Variants_fuzz_90'])


df_countries_fuzz_80,dict_countries_fuzz_80= pulisci_stringhe_fuzz(df_countries_fuzz_90,'State',80)
df_countries_fuzz_80= df_countries_fuzz_80.rename(columns={'State_mod':'State_fuzz_80'})
df_countries_fuzz_80['State_fuzz_80']=df_countries_fuzz_80['State_fuzz_80'].replace({'NY': 'NEW YORK', 'NC': 'NORTH CAROLINA'}) 
df_stati_corretti_fuzz_80=pd.DataFrame([(k, v) for k, v in dict_countries_fuzz_80.items() if len(v)>1], columns=['State', 'Variants_fuzz_80'])
#display(df_stati_corretti_fuzz_80_2)


df_countries_lev_1, dict_countries_lev_1=pulisci_stringhe_lev(df_countries_fuzz_80,'State',1) #applico la funzione lev
df_countries_lev_1= df_countries_lev_1.rename(columns={'State_mod':'State_lev_1'})
df_countries_lev_1['State_lev_1']=df_countries_lev_1['State_lev_1'].replace({'NY': 'NEW YORK', 'NC': 'NORTH CAROLINA'}) 
#df_countries_lev_1, guardando gli output, è quello che si avvicina di più ad essere corretto ma ha ancora dei problemi: NC diventa New York 

df_stati_corretti_lev_1=pd.DataFrame([(k, v) for k, v in dict_countries_lev_1.items() if len(v)>1], columns=['State', 'Variants_lev_1'])

df_merge_state_lev_fuzz90 = df_stati_corretti_fuzz_90.merge(df_stati_corretti_lev_1, on = 'State', how = 'outer')


df_merge_state_lev_fuzz90_fuzz80= df_merge_state_lev_fuzz90.merge(df_stati_corretti_fuzz_80, on='State', how='outer' )
display(df_merge_state_lev_fuzz90_fuzz80)


df_merge_state_lev_fuzz90_fuzz80.to_excel('/Users/alessandropavia/Desktop/Stage Reply/Sales Analysis 2/data/Countries_up.xlsx', sheet_name='Country', index=False)


In [ ]:
'''PASSO ALLA COLONNA CITY'''


df_countries_fuzz_city_90, dict_countries_fuzz_city_90= pulisci_stringhe_fuzz(df_countries_lev_1,'City',90) 
df_countries_fuzz_city_90 = df_countries_fuzz_city_90.rename(columns={'City_mod': 'City_fuzz_90'})
df_countries_city_corrette_fuzz_90=pd.DataFrame([(k, v) for k, v in dict_countries_fuzz_city_90.items() if len(v)>1], columns=['City', 'Variants_fuzz_90'])


df_countries_fuzz_city_95,dict_countries_fuzz_city_95= pulisci_stringhe_fuzz(df_countries_fuzz_city_90,'City',95) 
df_countries_fuzz_city_95 = df_countries_fuzz_city_95.rename(columns={'City_mod': 'City_fuzz_95'})
#display(df_countries_fuzz_city_95.loc[df_countries_fuzz_city_95['City'] == 'Amarillo']) #errori: bowling green viene scritto tutto attaccato
df_countries_fuzz_city_95['City_fuzz_95']= df_countries_fuzz_city_95['City_fuzz_95'].replace({'BOWLINGGREEN': 'BOWLING GREEN'}).replace({'WOODSTOK': 'WOODSTOCK'})
df_countries_city_corrette_fuzz_95=pd.DataFrame([(k, v) for k, v in dict_countries_fuzz_city_95.items() if len(v)>1], columns=['City', 'Variants_fuzz_95'])


df_merge_city_fuzz_95_fuzz_90=df_countries_city_corrette_fuzz_95.merge(df_countries_city_corrette_fuzz_90, on='City', how='outer')
#display(df_merge_city_fuzz_95_fuzz_90)


df_countries_lev_city_1, dict_countries_lev_city_1= pulisci_stringhe_lev(df_countries_fuzz_city_95,'City',1) 
df_countries_lev_city_1 = df_countries_lev_city_1.rename(columns={'City_mod': 'City_lev_1'})
#print(sorted(dict_countries_lev_city_1.items(), key=lambda couple: len(couple[1]), reverse=True))
df_countries_city_corrette_lev_1=pd.DataFrame([(k, v) for k, v in dict_countries_lev_city_1.items() if len(v)>1], columns=['City', 'Variants_lev_1'])

df_merge_city_fuzz_95_fuzz_90_lev_1=df_countries_city_corrette_lev_1.merge(df_merge_city_fuzz_95_fuzz_90, on='City', how='outer')
display(df_merge_city_fuzz_95_fuzz_90_lev_1)

with pd.ExcelWriter('/Users/alessandropavia/Desktop/Stage Reply/Sales Analysis 2/data/Countries_up.xlsx', engine='openpyxl', mode='a') as writer:
    df_merge_city_fuzz_95_fuzz_90_lev_1.to_excel(writer, sheet_name='City', index=False)

#METODO PIù GIUSTO: FUZZ CON SOGLIA=95 ci sono pochi errori in questa colonna city, ce ne sono molti nella colonna city del foglio orders, ancora da finire


#print(sorted(city_dict.items(), key=lambda couple: len(couple[1]), reverse=True))
df_countries_lev_city_1=df_countries_lev_city_1[['Country','City','City_fuzz_90','City_fuzz_95','City_lev_1','State','State_fuzz_90','State_fuzz_80','State_lev_1','Postal Code','Region']]


FOGLIO ORDERS

In [ ]:
df_orders_raw=df_orders
df_orders_mod=df_orders_raw.copy().drop_duplicates()

'''PARTO DALLA COLONNA SHIP MODE''' #posso usare metodo diverso
#display(df_orders_mod['Ship Mode'].unique())
#df_orders_mod['Ship Mode']=df_orders_mod['Ship Mode'].str.replace('1','First').str.replace('2','Second') 
df_orders_ship_mode_lev_1, dict_orders_ship_mode_lev_1 =pulisci_stringhe_lev(df_orders_mod,'Ship Mode',5)
#display(df_orders_ship_mode_lev_1['Ship Mode_mod'].unique()) #così ho pulito gli errori

print(dict_orders_ship_mode_lev_1)
df_orders_ship_mode_corrette=pd.DataFrame([(k, v) for k, v in dict_orders_ship_mode_lev_1.items() if len(v)>1], columns=['Ship Mode', 'Variants_lev_1'])
#display(df_orders_ship_mode_lev_1)
df_orders_ship_mode_corrette.to_excel('/Users/alessandropavia/Desktop/Stage Reply/Sales Analysis 2/data/Orders_up.xlsx', sheet_name='Ship Mode', index=False)


In [ ]:
'''PASSO ALLA COLONNA SALES'''
#print(df_orders_ship_mode_lev_1['Sales'].dtypes) #AGGIUNGERE DIZIONARIO
df_orders_ship_mode_lev_1['Sales_mod']= df_orders_ship_mode_lev_1['Sales'].apply(pulisci_trattino) # sostitiusco il - con il . quando è in mezzo alla stringa
df_orders_ship_mode_lev_1['Sales_mod'] = df_orders_ship_mode_lev_1['Sales_mod'].astype(str).str.strip() # 1. Rimuovi eventuali spazi bianchi e converti in string

df_orders_ship_mode_lev_1['Sales_mod'] = df_orders_ship_mode_lev_1['Sales_mod'].str.replace(",", '.', regex=False).str.replace("'", '.', regex=False).str.replace('a', '', regex=False) # 2. Sostituisci le virgole con punti (per uniformare la notazione decimale)

df_orders_ship_mode_lev_1['Sales_mod'] = pd.to_numeric(df_orders_ship_mode_lev_1['Sales_mod'], errors='coerce').round(2) 

dict_sales_mod = creo_dict(df_orders_ship_mode_lev_1,'Sales','Sales_mod')
print(dict_sales_mod)

df_sales_mod_corrette=pd.DataFrame([(k, v) for k, v in dict_sales_mod.items() if len(v)>1], columns=['Sales', 'Variants'])
with pd.ExcelWriter('/Users/alessandropavia/Desktop/Stage Reply/Sales Analysis 2/data/Orders_up.xlsx', engine='openpyxl', mode='a') as writer:
    df_sales_mod_corrette.to_excel(writer, sheet_name='Sales', index=False)


df_orders_ship_mode_lev_1=df_orders_ship_mode_lev_1[['Order ID','Order Date','Ship Date','Ship Mode','Ship Mode_mod','Customer ID','City','Postal Code','Product ID', 'Sales','Sales_mod','Quantity','Discount','Profit']] #avvicino Sales_mod a Sales


In [ ]:
'''PASSO ALLA COLONNA QUANTITY'''
#print(df_orders_ship_mode_lev_1['Quantity'].dtypes)
df_orders_ship_mode_lev_1['Quantity_mod']=df_orders_ship_mode_lev_1['Quantity'].astype(str).apply(pulisci_trattino)

df_orders_ship_mode_lev_1['Quantity_mod'] = df_orders_ship_mode_lev_1['Quantity_mod'].str.replace(",", '.', regex=False).str.replace("'", '.', regex=False).str.replace('ì', '', regex=False) # 2. Sostituisci le virgole con punti (per uniformare la notazione decimale)

df_orders_ship_mode_lev_1['Quantity_mod'] = pd.to_numeric(df_orders_ship_mode_lev_1['Quantity_mod'], errors='coerce')

dict_quantity_mod=creo_dict(df_orders_ship_mode_lev_1,'Quantity','Quantity_mod')

df_quantity_mod_corrette=pd.DataFrame([(k, v) for k, v in dict_quantity_mod.items() if len(v)>1], columns=['Quantity', 'Variants'])
display(df_quantity_mod_corrette)
with pd.ExcelWriter('/Users/alessandropavia/Desktop/Stage Reply/Sales Analysis 2/data/Orders_up.xlsx', engine='openpyxl', mode='a') as writer:
   df_quantity_mod_corrette.to_excel(writer, sheet_name='Quantity', index=False)

#display(df_orders_ship_mode_lev_1)
#display(df_orders_ship_mode_lev_1.loc[df_orders_ship_mode_lev_1['Quantity_mod'].isna()]) #-->pulita tutta la colonna Quantity_mod

In [ ]:
'''PASSO ALLA COLONNA DISCOUNT'''
df_orders_ship_mode_lev_1['Discount'].dtypes
df_orders_ship_mode_lev_1['Discount_mod'] = df_orders_ship_mode_lev_1['Discount'].astype(str).str.replace("ù", '', regex=False).str.replace("%", '', regex=False)
df_orders_ship_mode_lev_1['Discount_mod']=pd.to_numeric(df_orders_ship_mode_lev_1['Discount_mod'],errors='coerce')
dict_discount_mod=creo_dict(df_orders_ship_mode_lev_1,'Discount','Discount_mod')

df_discount_mod_corretto=pd.DataFrame([(k, v) for k, v in dict_discount_mod.items() if len(v)>1], columns=['Discount', 'Variants'])
display(df_discount_mod_corretto)
with pd.ExcelWriter('/Users/alessandropavia/Desktop/Stage Reply/Sales Analysis 2/data/Orders_up.xlsx', engine='openpyxl', mode='a') as writer:
   df_discount_mod_corretto.to_excel(writer, sheet_name='Discount', index=False)

#display(df_orders_ship_mode_lev_1.loc[df_orders_ship_mode_lev_1['Discount_mod'].isna()]) #così pulisco tutti gli errori



In [ ]:
'''PASSO ALLA COLONNA PROFIT'''

df_orders_ship_mode_lev_1['Profit_mod']= df_orders_ship_mode_lev_1['Profit'].astype(str).apply(pulisci_trattino) #in questo caso non metto .apply(pulisci_trattino) perché il profitto, a causa dello sconto, può essere negativo
df_orders_ship_mode_lev_1['Profit_mod'] = df_orders_ship_mode_lev_1['Profit_mod'].str.strip() # 1. Rimuovi eventuali spazi bianchi e converti in string
df_orders_ship_mode_lev_1['Profit_mod'] = df_orders_ship_mode_lev_1['Profit_mod'].str.replace(",", '.', regex=False).str.replace("'", '.', regex=False).str.replace('a', '', regex=False) # 2. Sostituisci le virgole con punti (per uniformare la notazione decimale)
df_orders_ship_mode_lev_1['Profit_mod'] = pd.to_numeric(df_orders_ship_mode_lev_1['Profit_mod'], errors='coerce').round(2)

dict_profit_mod=creo_dict(df_orders_ship_mode_lev_1,'Profit','Profit_mod')
df_profit_mod_corretto=pd.DataFrame([(k, v) for k, v in dict_profit_mod.items() if len(v)>1], columns=['Profit', 'Variants'])
display(df_profit_mod_corretto)
with pd.ExcelWriter('/Users/alessandropavia/Desktop/Stage Reply/Sales Analysis 2/data/Orders_up.xlsx', engine='openpyxl', mode='a') as writer:
   df_profit_mod_corretto.to_excel(writer, sheet_name='Profit', index=False)

df_orders_ship_mode_lev_1=df_orders_ship_mode_lev_1[['Order ID','Order Date','Ship Date','Ship Mode','Ship Mode_mod','Customer ID','City','Postal Code','Product ID', 'Sales','Sales_mod','Quantity','Quantity_mod','Discount','Discount_mod','Profit','Profit_mod']] 
#display(df_orders_ship_mode_lev_1.loc[df_orders_ship_mode_lev_1['Profit_mod'].isna()]) #-->pulita tutta la colonna Profit_mod


In [ ]:
'''PASSO ALLA COLONNA CITY DA FINIRe'''
#display(df_orders_ship_mode_lev_1['City'].sort_values().unique())
df_orders_fuzz_city_90, dict_orders_city_fuzz_90= pulisci_stringhe_fuzz(df_orders_ship_mode_lev_1,'City',90) 
df_orders_fuzz_city_90 = df_orders_fuzz_city_90.rename(columns={'City_mod': 'City_fuzz_90'})
df_orders_fuzz_city_90['City_fuzz_90']= df_orders_fuzz_city_90['City_fuzz_90'].replace({'BOWLINGGREEN': 'BOWLING GREEN'})
df_orders_city_corrette_fuzz_90=pd.DataFrame([(k, v) for k, v in dict_orders_city_fuzz_90.items() if len(v)>1], columns=['City', 'Variants_fuzz_90'])

df_orders_fuzz_city_85,dict_orders_city_fuzz_85=pulisci_stringhe_fuzz(df_orders_fuzz_city_90,'City',85) 
df_orders_fuzz_city_85 = df_orders_fuzz_city_85.rename(columns={'City_mod': 'City_fuzz_85'})
#print(df_orders_fuzz_city_95['City_fuzz_95'].sort_values().unique())#errori redding rimosso
df_orders_city_corrette_fuzz_85=pd.DataFrame([(k, v) for k, v in dict_orders_city_fuzz_85.items() if len(v)>1], columns=['City', 'Variants_fuzz_85'])

df_merge_orders_city_fuzz_90_fuzz_85=df_orders_city_corrette_fuzz_90.merge(df_orders_city_corrette_fuzz_85, on='City',how='outer')




df_orders_lev_city_1,dict_orders_city_lev_1=pulisci_stringhe_lev(df_orders_fuzz_city_85,'City',1) #forse quello con meno errori, ma ce ne sono ancora tanti(città che ha modificato ma non doveva: conroe, redding, camarillo, englewood, mason, medford, normal, layton, renton)
df_orders_lev_city_1 = df_orders_lev_city_1.rename(columns={'City_mod': 'City_lev_1'})
df_orders_city_corrette_lev_1=pd.DataFrame([(k, v) for k, v in dict_orders_city_lev_1.items() if len(v)>1], columns=['City', 'Variants_lev_1'])
df_merge_orders_city_fuzz_90_fuzz_85_lev_1=df_orders_city_corrette_lev_1.merge(df_merge_orders_city_fuzz_90_fuzz_85, on='City', how='outer')
df_orders_lev_city_1['City_lev_1']=df_orders_lev_city_1['City_lev_1'].replace({
    'ALECSANDRIA':'ALEXANDRIA',
    'LA':'LOS ANGELES',
    'BOWLINGGREEN': 'BOWLING GREEN',
    'MARLBOROUG':'MARLBOROUGH'
})
#print(sorted(dict_orders_city_lev_1.items(), key=lambda couple: len(couple[1]), reverse=True))


df_orders_lev_city_2,dict_orders_city_lev_2=pulisci_stringhe_lev(df_orders_lev_city_1,'City',2) #non va bene, rimuove troppe città corrette
df_orders_lev_city_2 = df_orders_lev_city_2.rename(columns={'City_mod': 'City_lev_2'})
df_orders_city_corrette_lev_2=pd.DataFrame([(k, v) for k, v in dict_orders_city_lev_2.items() if len(v)>1], columns=['City', 'Variants_lev_2'])
df_merge_orders_city_fuzz_90_fuzz_85_lev_1_lev_2=df_orders_city_corrette_lev_2.merge(df_merge_orders_city_fuzz_90_fuzz_85_lev_1, on='City', how='outer')
display(df_merge_orders_city_fuzz_90_fuzz_85_lev_1_lev_2)


with pd.ExcelWriter('/Users/alessandropavia/Desktop/Stage Reply/Sales Analysis 2/data/Orders_up.xlsx', engine='openpyxl', mode='a') as writer:
   df_merge_orders_city_fuzz_90_fuzz_85_lev_1_lev_2.to_excel(writer, sheet_name='City', index=False)


In [ ]:
def safe_parse_date(val):
    try:
        return pd.to_datetime(val, dayfirst=True)
    except:
        return val  # lascia il valore originale se fallisce




In [ ]:
def correggi_data(val1, val2):
    try:
        '''if isinstance(val1, str) and '-' in val1:
            val1_dt = pd.to_datetime(val1, errors='coerce')
            if pd.notnull(val1_dt):
                return val1_dt.strftime('%d/%m/%y')
        '''
        giorno1, mese1, anno1 = map(int, str(val1).split('/'))
        giorno2, mese2, anno2 = map(int, str(val2).split('/'))

        if anno1 < 1000:
            anno1 += 2000
        if anno2 < 1000:
            anno2 += 2000

        if mese1 > 12 or mese1==0:
            if mese2 > 1:
                nuovo_mese = mese2 - 1
                nuovo_anno = anno1
            else:
                nuovo_mese = 12
                nuovo_anno = anno2 - 1
            nuova_data = f"{giorno1:02d}/{nuovo_mese:02d}/{nuovo_anno}"
        else:
            nuova_data = f"{giorno1:02d}/{mese1:02d}/{anno1}"

        val1_dt = pd.to_datetime(nuova_data, format='%d/%m/%Y', errors='coerce')
        if pd.notnull(val1_dt):
            return val1_dt.strftime('%d/%m/%y')
        else:
            return None

    except Exception as e:
        print(f" Errore: {e} val1: {val1}, val2: {val2}")
        return None


In [ ]:
def pulisci_date(df, colonna1, colonna2):
    df2 = df.copy()

    colonna1_mod = f'{colonna1}_mod'
    colonna2_mod = f'{colonna2}_mod'

    df2[colonna1_mod] = df2[colonna1].apply(safe_parse_date) #applico pd.to_datetime, se non funziona restituisce il valore originale
    df2[colonna2_mod] = df2[colonna2].apply(safe_parse_date)
    

    # Itera per eventuali correzioni logiche
    for index, row in df2.iterrows():
        val1 = row[colonna1_mod]
        val2 = row[colonna2_mod]

        if pd.notnull(val1) and pd.notnull(val2) and isinstance(val1, pd.Timestamp) and isinstance(val2, pd.Timestamp): #se è stato moidificato in timestamp allora controllo
            giorno1, mese1, anno1 = val1.day, val1.month, val1.year
            giorno2, mese2, anno2 = val2.day, val2.month, val2.year

            if mese1 > 12:
            # Tenta di usare lo stesso mese della data 2 (Ship Date)
                try:
                    nuova_data = Timestamp(day=giorno1, month=mese2, year=anno1)
                    if nuova_data <= val2:
                        df2.at[index, colonna1_mod] = nuova_data
                    else:
                        # Se la data supera la Ship Date, prova con mese precedente
                        if mese2 > 1:
                            nuova_data = Timestamp(day=giorno1, month=mese2 - 1, year=anno1)
                            df2.at[index, colonna1_mod] = nuova_data
                        else:
                            # se order date> ship date e ship date si trova a gennaio, allora l'order date rimane lo stesso giorno, del mese di dicembre dell'anno prima rispetto all'anno della spedizione
                            nuova_data = Timestamp(day=giorno1, month=12, year=anno2 - 1)
                            df2.at[index, colonna1_mod] = nuova_data
                except ValueError as e:
                    print(f"Errore alla riga {index}: {e}")
        
        else:
            data_corretta = correggi_data(val1, val2)
            if data_corretta:
                df2.at[index, colonna1_mod] = data_corretta


    
    df2[colonna1_mod] = pd.to_datetime(df2[colonna1_mod], errors='coerce')
    df2[colonna2_mod] = pd.to_datetime(df2[colonna2_mod], errors='coerce')



    return df2


In [ ]:



'''COLONNA ORDER DATE'''

df_order_date=pulisci_date(df_orders_ship_mode_lev_1,'Order Date','Ship Date')
#display(df_order_date)
righe_non_convertite_order1 = df_order_date[df_order_date['Order Date_mod'].isna()]
print(f"Righe non convertite: {len(righe_non_convertite_order1)}")
display(df_order_date.loc[df_order_date['Order Date_mod'].isna()])
#AGGIUNGERE DIZIONARIO


'''
# 1. Converte qualsiasi stringa interpretabile come data

df_orders_ship_mode_lev_1['Order Date_mod'] = pd.to_datetime(df_orders_ship_mode_lev_1['Order Date'], errors='coerce', dayfirst=True)

# 2. Elimina o segnala i casi non convertiti (rimasti NaT)
righe_non_convertite_order = df_orders_ship_mode_lev_1[df_orders_ship_mode_lev_1['Order Date_mod'].isna()]
print(f"Righe non convertite: {len(righe_non_convertite_order)}")

# 3. Formatta le date convertite nel formato giorno/mese/anno
df_orders_ship_mode_lev_1['order Date_mod'] = df_orders_ship_mode_lev_1['Order Date_mod'].dt.strftime('%d/%m/%y')

#df_orders_ship_mode_lev_1=df_orders_ship_mode_lev_1[['Order ID','Order Date','Order Date_mod','Ship Date','Ship Mode','Ship Mode_mod','Customer ID','City','Postal Code','Product ID', 'Sales','Sales_mod','Quantity','Quantity_mod','Discount','Discount_mod','Profit','Profit_mod']] 
display(df_orders_ship_mode_lev_1.loc[df_orders_ship_mode_lev_1['Order Date_mod'].isna()])

'''

In [ ]:
'''COLONNA SHIP DATE'''
# 1. Converte qualsiasi stringa interpretabile come data
df_orders_ship_mode_lev_1['Ship Date_mod'] = pd.to_datetime(df_orders_ship_mode_lev_1['Ship Date'], errors='coerce', dayfirst=True)

# 2. Elimina o segnala i casi non convertiti (rimasti NaT)
righe_non_convertite_ship = df_orders_ship_mode_lev_1[df_orders_ship_mode_lev_1['Ship Date_mod'].isna()]
print(f"Righe non convertite: {len(righe_non_convertite_ship)}")

# 3. Formatta le date convertite nel formato giorno/mese/anno
df_orders_ship_mode_lev_1['Ship Date_mod'] = df_orders_ship_mode_lev_1['Ship Date_mod'].dt.strftime('%d/%m/%y')

df_orders_ship_mode_lev_1=df_orders_ship_mode_lev_1[['Order ID','Order Date','Order Date_mod','Ship Date','Ship Date_mod','Ship Mode','Ship Mode_mod','Customer ID','City','Postal Code','Product ID', 'Sales','Sales_mod','Quantity','Quantity_mod','Discount','Discount_mod','Profit','Profit_mod']] 
display(df_orders_ship_mode_lev_1.loc[df_orders_ship_mode_lev_1['Ship Date_mod'].isna()])

FOGLIO PRODUCTS

In [ ]:
df_products_raw = df_products
df_products_mod=df_products_raw.copy().drop_duplicates()


'''COLONNA CATEGORY'''
#print(df_products_mod['Category'].unique())
df_products_cat_lev_6, dict_products_cat=pulisci_stringhe_lev(df_products_mod,'Category',6)
#display(df_products_cat_lev_6['Category_mod'].unique()) #ok, così pulisce tutti gli errori
df_products_cat_corrette=pd.DataFrame([(k, v) for k, v in dict_products_cat.items() if len(v)>1], columns=['Category', 'Variants'])
display(df_products_cat_corrette)
df_products_cat_corrette.to_excel('/Users/alessandropavia/Desktop/Stage Reply/Sales Analysis 2/data/Products_up.xlsx', sheet_name='Category', index=False)

'''COLONNA SUB-CATEGORY'''
#display(df_products_lev_6['Sub-Category'].unique())
df_products_sub_lev2, dict_products_sub=pulisci_stringhe_lev(df_products_cat_lev_6,'Sub-Category',2)
#display(df_products_sub_lev2['Sub-Category_mod'].unique()) ok, così pulisce tutti gli errori
df_products_sub_corrette=pd.DataFrame([(k, v) for k, v in dict_products_sub.items() if len(v)>1], columns=['Sub-Category', 'Variants'])
with pd.ExcelWriter('/Users/alessandropavia/Desktop/Stage Reply/Sales Analysis 2/data/Products_up.xlsx', engine='openpyxl', mode='a') as writer:
   df_products_sub_corrette.to_excel(writer, sheet_name='Sub-Category', index=False)

df_products_sub_lev2=df_products_sub_lev2[['Product ID','Category','Category_mod','Sub-Category','Sub-Category_mod','Product Name']]#cambio ordine delle colonnne


'''COLONNA PRODUCT-ID'''
df_products_sub_lev2['Product ID_mod']=df_products_sub_lev2['Product ID']

for index, row in df_products_sub_lev2.iterrows():
    product_id = row['Product ID_mod']
    category = row['Category_mod']
    sub_category= row['Sub-Category_mod']

    if pd.notnull(product_id) and '-' in product_id:
        product_cat = product_id.split('-')[0]
        product_sub_cat = product_id.split('-')[1]

        if product_cat != category[:3]:  # primi 3 caratteri della stringa
            print(f'La riga {index} è sbagliata: ID = {product_id}, Categoria = {category}')
        
        elif product_sub_cat != sub_category[:2]:
            print(f'La riga {index} è sbagliata: ID= {product_id}, Sub-Category= {product_sub_cat}')
            #OK NO ERRORI





FOGLIO CUSTOMERS

In [ ]:
df_customers_raw=df_customers
df_customers_mod=df_customers_raw.copy().drop_duplicates()

'''COLONNA SEGMENT'''
#display(df_customers_mod['Segment'].unique())
df_customers_lev_2, dict_segment=pulisci_stringhe_lev(df_customers_mod,'Segment',2)
#display(df_customers_lev_2['Segment_mod'].unique())#ok, cosi pulisce tutti gli errori

df_customers_segment_corretto=pd.DataFrame([(k, v) for k, v in dict_segment.items() if len(v)>1], columns=['Segment', 'Variants'])
df_customers_segment_corretto.to_excel('/Users/alessandropavia/Desktop/Stage Reply/Sales Analysis 2/data/Customers_up.xlsx', sheet_name='Segment', index=False)

df_customers_lev_2['Customer ID_mod']=df_customers_lev_2['Customer ID'].copy()
df_customers_lev_2['Customer Name_mod'] = (df_customers_lev_2['Customer Name'].str.replace('-', ' ', regex=False).str.replace('_', ' ', regex=False))

'''COLONNA CUSTOMER ID'''
for index in df_customers_lev_2.index:
    customer_id = df_customers_lev_2.at[index, 'Customer ID_mod']
    customer_full_name = df_customers_lev_2.at[index, 'Customer Name_mod']
    
    if pd.notnull(customer_id) and '-' in customer_id:
        customer_id_first_letter_name = customer_id[0]
        customer_id_first_letter_surname = customer_id[1]

        parts = customer_full_name.strip().split()
        if len(parts) >= 2:
            first_letter_name = parts[0][0]
            first_letter_surname = parts[1][0]
        else:
            # Gestione nome-cognome attaccati tipo 'CoreyLock'
            split_parts = re.split(r'(?=[A-Z])', customer_full_name)
            if len(split_parts) >= 3:
                first_letter_name = split_parts[1][0]
                first_letter_surname = split_parts[2][0]
            else:
                continue  # nome non gestibile, salta

        if (
            customer_id_first_letter_name != first_letter_name or
            customer_id_first_letter_surname != first_letter_surname
        ):
            print(f'La riga {index} è errata: il Customer ID è {customer_id}, il Customer Name è {customer_full_name}')
            df_customers_lev_2.at[index, 'Customer ID_mod'] = first_letter_name + first_letter_surname + customer_id[2:]

            

dict_customers_id_corretti=creo_dict(df_customers_lev_2,'Customer ID','Customer ID_mod')
df_customers_id_corretti=pd.DataFrame([(k, v) for k, v in dict_customers_id_corretti.items()], columns=['Customer ID', 'Variants'])
display(df_customers_id_corretti.loc[df_customers_id_corretti['Customer ID']=='CL-12640'])
with pd.ExcelWriter('/Users/alessandropavia/Desktop/Stage Reply/Sales Analysis 2/data/Customers_up.xlsx', engine='openpyxl', mode='a') as writer:
   df_customers_id_corretti.to_excel(writer, sheet_name='Customer ID', index=False)
display(df_customers_lev_2.loc[df_customers_lev_2['Customer Name_mod']=='Corey Lock']) #ok a posto
    